<a href="https://colab.research.google.com/github/judebebo32/ML_CLG/blob/main/Cedluloselignindata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models


In [2]:
data = pd.read_excel('/content/New_data_2 - Copy.xlsx')
x = data.iloc[:, 0:8].values
y = data.iloc[:, 8:11].values

In [3]:
# Define generator model
def make_generator_model():
    model = models.Sequential()
    model.add(layers.Dense(16, input_shape=(8,), activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(3))
    return model

# Define discriminator model
def make_discriminator_model():
    model = models.Sequential()
    model.add(layers.Dense(32, input_shape=(3,), activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [4]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [5]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Define discriminator loss function
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Define generator loss function
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [6]:
# Define training loop
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake data using the generator
        x_fake = generator(noise, training=True)

        # Get discriminator predictions for real and fake data
        y_real_pred = discriminator(y_real, training=True)
        y_fake_pred = discriminator(x_fake, training=True)

        # Calculate losses
        gen_loss = generator_loss(y_fake_pred)
        disc_loss = discriminator_loss(y_real_pred, y_fake_pred)

    # Calculate gradients and apply to optimizer
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [8]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0 - Generator Loss: 0.6985, Discriminator Loss: 0.6893
Epoch 50 - Generator Loss: 0.7828, Discriminator Loss: 0.6167
Epoch 100 - Generator Loss: 1.0039, Discriminator Loss: 0.4697
Epoch 150 - Generator Loss: 1.3365, Discriminator Loss: 0.3240
Epoch 200 - Generator Loss: 1.7365, Discriminator Loss: 0.2049
Epoch 250 - Generator Loss: 2.1377, Discriminator Loss: 0.1460
Epoch 300 - Generator Loss: 2.4334, Discriminator Loss: 0.1290
Epoch 350 - Generator Loss: 2.3611, Discriminator Loss: 0.2374
Epoch 400 - Generator Loss: 2.3314, Discriminator Loss: 0.3781
Epoch 450 - Generator Loss: 2.1357, Discriminator Loss: 0.5853
Epoch 500 - Generator Loss: 1.9513, Discriminator Loss: 0.9598
Epoch 550 - Generator Loss: 2.0656, Discriminator Loss: 1.0452
Epoch 600 - Generator Loss: 1.8851, Discriminator Loss: 1.0259
Epoch 650 - Generator Loss: 1.3433, Discriminator Loss: 1.3849
Epoch 700 - Generator Loss: 1.1969, Discriminator Loss: 1.1425
Epoch 750 - Generator Loss: 1.2081, Discriminator Loss: 1.

In [12]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data using the trained generator model
    num_samples = len(x_val)
    noise = tf.random.normal([num_samples, 8])
    synthetic_data = generator(noise).numpy()

    # Convert the synthetic data to a pandas DataFrame
    synthetic_df = pd.DataFrame(np.hstack((x_val, synthetic_data)), columns=['Input 1', 'Input 2', 'Input 3', 'Input 4', 'Input 5', 'Input 6', 'Input 7', 'Input 8', 'Output 1', 'Output 2', 'Output 3'])

   


Epoch 0 - Generator Loss: 0.8369, Discriminator Loss: 1.4098
Epoch 50 - Generator Loss: 0.8621, Discriminator Loss: 1.3722
Epoch 100 - Generator Loss: 0.8484, Discriminator Loss: 1.3699
Epoch 150 - Generator Loss: 0.9010, Discriminator Loss: 1.3354
Epoch 200 - Generator Loss: 0.8586, Discriminator Loss: 1.3836
Epoch 250 - Generator Loss: 0.9117, Discriminator Loss: 1.3494
Epoch 300 - Generator Loss: 0.8457, Discriminator Loss: 1.4319
Epoch 350 - Generator Loss: 0.8648, Discriminator Loss: 1.3494
Epoch 400 - Generator Loss: 0.8746, Discriminator Loss: 1.3292
Epoch 450 - Generator Loss: 0.8901, Discriminator Loss: 1.3665
Epoch 500 - Generator Loss: 0.8950, Discriminator Loss: 1.3539
Epoch 550 - Generator Loss: 0.9205, Discriminator Loss: 1.3436
Epoch 600 - Generator Loss: 0.8292, Discriminator Loss: 1.3444
Epoch 650 - Generator Loss: 0.8677, Discriminator Loss: 1.3565
Epoch 700 - Generator Loss: 0.8346, Discriminator Loss: 1.3659
Epoch 750 - Generator Loss: 0.8042, Discriminator Loss: 1.

In [24]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
fold = 1

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data using the trained generator model
    num_samples = len(x_val)
    noise = tf.random.normal([num_samples, 8])
    synthetic_data = generator(noise).numpy()

    # Convert the synthetic data to a pandas DataFrame
    synthetic_df = pd.DataFrame(np.hstack((x_val, synthetic_data)), columns=['Input 1', 'Input 2', 'Input 3', 'Input 4', 'Input 5', 'Input 6', 'Input 7', 'Input 8', 'Output 1', 'Output 2', 'Output 3'])

   # Export the synthetic data to an Excel file
    synthetic_df.to_excel(f'/content/sample_data/synthetic_data_fold{fold}.xlsx', index=False)

    fold += 1

Epoch 0 - Generator Loss: 0.9367, Discriminator Loss: 1.5389
Epoch 50 - Generator Loss: 0.8132, Discriminator Loss: 1.4520
Epoch 100 - Generator Loss: 0.8368, Discriminator Loss: 1.3477
Epoch 150 - Generator Loss: 0.8259, Discriminator Loss: 1.4658
Epoch 200 - Generator Loss: 0.8713, Discriminator Loss: 1.3554
Epoch 250 - Generator Loss: 0.8171, Discriminator Loss: 1.4874
Epoch 300 - Generator Loss: 0.8269, Discriminator Loss: 1.4089
Epoch 350 - Generator Loss: 0.8643, Discriminator Loss: 1.4313
Epoch 400 - Generator Loss: 0.8893, Discriminator Loss: 1.3756
Epoch 450 - Generator Loss: 0.8554, Discriminator Loss: 1.4547
Epoch 500 - Generator Loss: 0.8672, Discriminator Loss: 1.4254
Epoch 550 - Generator Loss: 0.8904, Discriminator Loss: 1.4568
Epoch 600 - Generator Loss: 0.8542, Discriminator Loss: 1.4223
Epoch 650 - Generator Loss: 0.9550, Discriminator Loss: 1.4103
Epoch 700 - Generator Loss: 0.8323, Discriminator Loss: 1.4833
Epoch 750 - Generator Loss: 0.8505, Discriminator Loss: 1.

In [21]:
import matplotlib.pyplot as plt

# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
epoch_num = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

        epoch_num.append(epoch)



Epoch 0 - Generator Loss: 0.7286, Discriminator Loss: 1.5904
Epoch 50 - Generator Loss: 0.7056, Discriminator Loss: 1.4121
Epoch 100 - Generator Loss: 0.7326, Discriminator Loss: 1.4734
Epoch 150 - Generator Loss: 0.7251, Discriminator Loss: 1.4260
Epoch 200 - Generator Loss: 0.6977, Discriminator Loss: 1.4606
Epoch 250 - Generator Loss: 0.6520, Discriminator Loss: 1.4358
Epoch 300 - Generator Loss: 0.7916, Discriminator Loss: 1.4289
Epoch 350 - Generator Loss: 0.7452, Discriminator Loss: 1.4365
Epoch 400 - Generator Loss: 0.7900, Discriminator Loss: 1.3846
Epoch 450 - Generator Loss: 0.8258, Discriminator Loss: 1.4362
Epoch 500 - Generator Loss: 0.8872, Discriminator Loss: 1.3950
Epoch 550 - Generator Loss: 0.8615, Discriminator Loss: 1.4160
Epoch 600 - Generator Loss: 0.8369, Discriminator Loss: 1.4191
Epoch 650 - Generator Loss: 0.8074, Discriminator Loss: 1.3946
Epoch 700 - Generator Loss: 0.8760, Discriminator Loss: 1.4034
Epoch 750 - Generator Loss: 0.8480, Discriminator Loss: 1.

In [23]:
import pandas as pd

# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    mae_fold = {'Fold': fold}
    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

        mae = mae.append(mae_fold, ignore_index=True)

# Export MAE data to Excel file
mae.to_excel('/content/sample_data/mae_data.xlsx', index=False)

Fold 1, Epoch 0 - Generator Loss: 0.8493, Discriminator Loss: 1.6182


<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 0.8521, Discriminator Loss: 1.3495


<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 100 - Generator Loss: 0.8707, Discriminator Loss: 1.3757


...

<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 5, Epoch 900 - Generator Loss: 0.8581, Discriminator Loss: 1.4257


<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 5, Epoch 950 - Generator Loss: 0.8428, Discriminator Loss: 1.3785


<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-23-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

# New Section

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# Load data from Excel file
data = pd.read_excel('/content/New_data_2 - Copy.xlsx')

# Split input and output variables
x = data.iloc[:, :8].values
y = data.iloc[:, 8:].values

# Define generator and discriminator models
def make_generator_model():
    model = keras.Sequential([
        keras.layers.Dense(16, input_shape=(8,), activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(3, activation='linear')
    ])
    return model

def make_discriminator_model():
    model = keras.Sequential([
        keras.layers.Dense(16, input_shape=(3,), activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define GAN model
def make_gan_model(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential([
        generator,
        discriminator
    ])
    return model

# Define loss functions
binary_crossentropy = keras.losses.BinaryCrossentropy(from_logits=False)
mse = keras.losses.MeanSquaredError()

# Define optimizers
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Define training step for GAN model
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        x_fake = generator(noise)
        y_fake = discriminator(x_fake)
        y_real_pred = discriminator(y_real)

        gen_loss = mse(y_real, x_fake)
        disc_loss_fake = binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        disc_loss_real = binary_crossentropy(tf.ones_like(y_real_pred), y_real_pred)
        disc_loss = (disc_loss_fake + disc_loss_real) / 2

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss


# New Section

In [18]:
!ls /content/sample_data/

anscombe.json		      synthetic_data_fold1.xlsx
california_housing_test.csv   synthetic_data_fold2.xlsx
california_housing_train.csv  synthetic_data_fold3.xlsx
mnist_test.csv		      synthetic_data_fold4.xlsx
mnist_train_small.csv	      synthetic_data_fold5.xlsx
README.md


In [30]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    generator = make_generator_model()
    discriminator = make_discriminator_model()
    gan = make_gan_model(generator, discriminator)

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        mae_fold = {'Fold': fold}
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()
        mae = mae.append(mae_fold, ignore_index=True)

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data and export to Excel file
    synthetic_data = generator(tf.random.normal([len(x), 8])).numpy()
    synthetic_df = pd.DataFrame(np.concatenate([x, synthetic_data], axis=1), columns=data.columns)
    synthetic_df.to_excel(f'/content/sample_data_fold{fold}.xlsx', index=False)

# Export MAE data to Excel file
mae.to_excel('/content/sample_data.xlsx', index=False)

<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 0 - Generator Loss: 645.0358, Discriminator Loss: 0.3554


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 99.3816, Discriminator Loss: 2.2384


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

...

Fold 2, Epoch 900 - Generator Loss: 54.6975, Discriminator Loss: 3.4066


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 2, Epoch 950 - Generator Loss: 55.5560, Discriminator Loss: 3.4188


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

KeyError: ignored

In [31]:
# Define optimizers
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Define training step for GAN model
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        x_fake = generator(noise)
        y_fake = discriminator(x_fake)
        y_real_pred = discriminator(y_real)

        gen_loss = mse(y_real, x_fake)
        disc_loss_fake = binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        disc_loss_real = binary_crossentropy(tf.ones_like(y_real_pred), y_real_pred)
        disc_loss = (disc_loss_fake + disc_loss_real) / 2

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.build(generator.trainable_variables)
    discriminator_optimizer.build(discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [32]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    generator = make_generator_model()
    discriminator = make_discriminator_model()
    gan = make_gan_model(generator, discriminator)

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        mae_fold = {'Fold': fold}
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()
        mae = mae.append(mae_fold, ignore_index=True)

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 0 - Generator Loss: 639.7891, Discriminator Loss: 0.3483


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 97.2751, Discriminator Loss: 2.5994


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

...

Fold 2, Epoch 900 - Generator Loss: 54.8442, Discriminator Loss: 1.8182


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 2, Epoch 950 - Generator Loss: 55.9813, Discriminator Loss: 1.8298


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

KeyError: ignored